# Lab 1.1.5: Ollama Benchmarking

**Module:** 1.1 - DGX Spark Platform Mastery  
**Time:** 2 hours  
**Difficulty:** ⭐⭐⭐ (Intermediate-Advanced)

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Understand LLM inference metrics (prefill vs decode)
- [ ] Use Ollama's API for accurate benchmarking
- [ ] Benchmark multiple model sizes (3B, 8B, 70B)
- [ ] Compare your results with NVIDIA published specs

---

## 📚 Prerequisites

- Completed: Tasks 1.1-1.4
- Ollama installed and running
- At least one model pulled (llama3.2:3b recommended to start)

---

## 🌍 Real-World Context

When choosing an LLM for your application, speed matters:
- **Chatbots** need low latency for good user experience
- **Batch processing** needs high throughput to be cost-effective
- **Edge deployment** has strict memory constraints

Understanding how to benchmark helps you make informed decisions about model selection and hardware utilization.

---

## 🧒 ELI5: LLM Inference Metrics

> **Imagine you're reading a question and writing an answer...**
>
> **Prefill (Prompt Processing)**: This is like reading and understanding the question. You need to read ALL the words before you can start answering. Speed = how fast you can read.
>
> **Decode (Token Generation)**: This is like writing your answer, one word at a time. Each word depends on what you wrote before. Speed = how fast you can write.
>
> **Tokens**: Words or pieces of words. "Hello world" = 2 tokens. "Unbelievable" might be 3 tokens ("Un", "believ", "able").
>
> **In AI terms:**
> - Prefill (tok/s): How fast the model processes your input prompt
> - Decode (tok/s): How fast the model generates new tokens (the response)
>
> Prefill is usually much faster because we can process many tokens in parallel. Decode is slower because each token depends on all previous tokens.

---

## Part 1: Verify Ollama Setup

### Checking Ollama Service

In [ ]:
import requests
import json
import time
from typing import Dict, List, Optional

# Ollama API configuration - update if using a different host/port
OLLAMA_BASE_URL: str = "http://localhost:11434"

def check_ollama_status(base_url: str = OLLAMA_BASE_URL) -> bool:
    """
    Check if Ollama is running and accessible.
    
    Args:
        base_url: Ollama API base URL
        
    Returns:
        True if Ollama is accessible, False otherwise
    """
    try:
        response = requests.get(f"{base_url}/api/tags", timeout=5)
        if response.status_code == 200:
            print("Ollama is running!")
            return True
        else:
            print(f"Ollama responded with status {response.status_code}")
            return False
    except requests.exceptions.ConnectionError:
        print("Cannot connect to Ollama!")
        print("   Start it with: ollama serve")
        print("   Or check: systemctl status ollama")
        return False
    except Exception as e:
        print(f"Error: {e}")
        return False

check_ollama_status()

In [ ]:
# List available models
def list_models() -> List[str]:
    """
    List models available in Ollama.
    
    Returns:
        List of model names, or empty list if none found or error.
    """
    try:
        response = requests.get(f"{OLLAMA_BASE_URL}/api/tags", timeout=10)
        response.raise_for_status()  # Raise exception for bad status codes
        data = response.json()
        models = data.get("models", [])
        
        if not models:
            print("No models found. Pull some models first:")
            print("  ollama pull llama3.2:3b")
            print("  ollama pull llama3.1:8b")
            print("  ollama pull llama3.1:70b")
            return []
        
        print("Available Models:")
        print("=" * 60)
        print(f"{'Name':<35} {'Size':<15} {'Modified'}")
        print("-" * 60)
        
        for model in models:
            name = model.get("name", "unknown")
            size_bytes = model.get("size", 0)
            size_gb = size_bytes / (1024**3)
            modified = model.get("modified_at", "")[:10]
            print(f"{name:<35} {size_gb:.1f} GB{'':<8} {modified}")
        
        return [m["name"] for m in models]
        
    except requests.exceptions.ConnectionError:
        print("❌ Cannot connect to Ollama!")
        print("   Start it with: ollama serve")
        print("   Or check: systemctl status ollama")
        return []
    except requests.exceptions.Timeout:
        print("❌ Request timed out - Ollama may be busy")
        return []
    except requests.exceptions.RequestException as e:
        print(f"❌ Request error: {e}")
        return []
    except Exception as e:
        print(f"❌ Error listing models: {e}")
        return []

available_models = list_models()

### ✋ Try It Yourself #1

If you don't have models yet, pull them now. Run in a terminal:

```bash
# Start with the smallest model
ollama pull llama3.2:3b

# Medium model (8 billion parameters)
ollama pull llama3.1:8b

# Large model (70 billion parameters) - only if you have enough storage!
ollama pull llama3.1:70b
```

---

## Part 2: Understanding the API Response

### Key Metrics from Ollama

Ollama's `/api/generate` endpoint returns timing information in nanoseconds:

| Field | Description |
|-------|-------------|
| `prompt_eval_count` | Number of tokens in the prompt |
| `prompt_eval_duration` | Time to process prompt (nanoseconds) |
| `eval_count` | Number of tokens generated |
| `eval_duration` | Time to generate response (nanoseconds) |
| `total_duration` | Total request time (nanoseconds) |

In [ ]:
def single_inference(model: str, prompt: str, max_tokens: int = 50) -> Dict:
    """
    Run a single inference and return raw timing data.
    
    Args:
        model: Model name (e.g., "llama3.2:3b")
        prompt: Input prompt
        max_tokens: Maximum tokens to generate
    
    Returns:
        Dictionary with timing metrics
    """
    try:
        response = requests.post(
            f"{OLLAMA_BASE_URL}/api/generate",
            json={
                "model": model,
                "prompt": prompt,
                "stream": False,
                "options": {
                    "num_predict": max_tokens
                }
            },
            timeout=300
        )
        
        if response.status_code != 200:
            return {"error": f"Status {response.status_code}: {response.text[:100]}"}
        
        return response.json()
    
    except requests.exceptions.Timeout:
        return {"error": "Request timed out (300s limit)"}
    except requests.exceptions.ConnectionError:
        return {"error": "Cannot connect to Ollama - is it running?"}
    except requests.exceptions.RequestException as e:
        return {"error": f"Request failed: {e}"}
    except Exception as e:
        return {"error": f"Unexpected error: {type(e).__name__}: {e}"}

# Test with a simple prompt
if available_models:
    test_model = available_models[0]
    print(f"Testing with {test_model}...\n")
    
    result = single_inference(test_model, "Hello, how are you?", max_tokens=20)
    
    if "error" not in result:
        print("Response:", result.get("response", "")[:100])
        print("\nRaw timing data:")
        print(f"  prompt_eval_count:    {result.get('prompt_eval_count', 0)} tokens")
        print(f"  prompt_eval_duration: {result.get('prompt_eval_duration', 0) / 1e9:.3f} seconds")
        print(f"  eval_count:           {result.get('eval_count', 0)} tokens")
        print(f"  eval_duration:        {result.get('eval_duration', 0) / 1e9:.3f} seconds")
        print(f"  total_duration:       {result.get('total_duration', 0) / 1e9:.3f} seconds")
    else:
        print(f"Error: {result['error']}")
else:
    print("No models available. Pull a model first!")

---

## Part 3: Benchmark Utility Functions

> **Note:** A more complete version of these utilities is available in `utils/benchmark_utils.py`.
> For production use, you can import: `from utils.benchmark_utils import OllamaBenchmark, BenchmarkResult, quick_benchmark`

In [ ]:
import statistics
from dataclasses import dataclass
from datetime import datetime

@dataclass
class BenchmarkResult:
    """Container for benchmark results."""
    model: str
    prompt_tokens: int
    generated_tokens: int
    prefill_tps: float  # tokens per second (prompt processing)
    decode_tps: float   # tokens per second (generation)
    total_time_s: float
    memory_gb: float = 0.0
    
    def __str__(self):
        return (
            f"{self.model}: "
            f"Prefill: {self.prefill_tps:.1f} tok/s, "
            f"Decode: {self.decode_tps:.1f} tok/s, "
            f"Memory: {self.memory_gb:.1f} GB"
        )

def get_gpu_memory() -> float:
    """Get GPU memory usage in GB."""
    import subprocess
    try:
        result = subprocess.run(
            ["nvidia-smi", "--query-gpu=memory.used", "--format=csv,noheader,nounits"],
            capture_output=True, text=True, timeout=5
        )
        return float(result.stdout.strip()) / 1024  # MiB to GB
    except:
        return 0.0

def benchmark_model(
    model: str,
    prompt: str = "Explain the concept of machine learning in simple terms.",
    max_tokens: int = 100,
    num_runs: int = 3,
    warmup: bool = True
) -> BenchmarkResult:
    """
    Benchmark a model with multiple runs for accuracy.
    
    Args:
        model: Model name
        prompt: Test prompt
        max_tokens: Tokens to generate
        num_runs: Number of runs to average
        warmup: Whether to do warmup run
    
    Returns:
        BenchmarkResult with averaged metrics
    """
    # Warmup run (GPU needs to "wake up")
    if warmup:
        print(f"  Warming up {model}...")
        single_inference(model, "Hello", max_tokens=5)
    
    # Get memory usage after model is loaded
    memory_gb = get_gpu_memory()
    
    # Collect metrics from multiple runs
    prefill_speeds = []
    decode_speeds = []
    total_times = []
    prompt_tokens = 0
    gen_tokens = []
    
    print(f"  Running {num_runs} benchmark iterations...")
    
    for i in range(num_runs):
        result = single_inference(model, prompt, max_tokens)
        
        if "error" in result:
            print(f"    Run {i+1}: ERROR - {result['error']}")
            continue
        
        # Extract timing (convert ns to seconds)
        prompt_count = result.get("prompt_eval_count", 0)
        prompt_duration = result.get("prompt_eval_duration", 0) / 1e9
        eval_count = result.get("eval_count", 0)
        eval_duration = result.get("eval_duration", 0) / 1e9
        total_duration = result.get("total_duration", 0) / 1e9
        
        # Calculate tokens per second
        prefill_tps = prompt_count / prompt_duration if prompt_duration > 0 else 0
        decode_tps = eval_count / eval_duration if eval_duration > 0 else 0
        
        prefill_speeds.append(prefill_tps)
        decode_speeds.append(decode_tps)
        total_times.append(total_duration)
        prompt_tokens = prompt_count
        gen_tokens.append(eval_count)
        
        print(f"    Run {i+1}: Prefill: {prefill_tps:.1f} tok/s, Decode: {decode_tps:.1f} tok/s")
    
    if not prefill_speeds:
        return BenchmarkResult(
            model=model,
            prompt_tokens=0,
            generated_tokens=0,
            prefill_tps=0,
            decode_tps=0,
            total_time_s=0,
            memory_gb=memory_gb
        )
    
    # Average the results
    return BenchmarkResult(
        model=model,
        prompt_tokens=prompt_tokens,
        generated_tokens=int(statistics.mean(gen_tokens)),
        prefill_tps=statistics.mean(prefill_speeds),
        decode_tps=statistics.mean(decode_speeds),
        total_time_s=statistics.mean(total_times),
        memory_gb=memory_gb
    )

print("Benchmark functions defined!")

---

## Part 4: Run Benchmarks

### Single Model Benchmark

In [ ]:
# Benchmark a single model
if available_models:
    test_model = available_models[0]
    print(f"\n{'='*60}")
    print(f"Benchmarking: {test_model}")
    print(f"{'='*60}")
    
    result = benchmark_model(test_model, num_runs=3)
    
    print(f"\n📊 Results:")
    print(f"  Prompt tokens:     {result.prompt_tokens}")
    print(f"  Generated tokens:  {result.generated_tokens}")
    print(f"  Prefill speed:     {result.prefill_tps:.1f} tokens/second")
    print(f"  Decode speed:      {result.decode_tps:.1f} tokens/second")
    print(f"  Total time:        {result.total_time_s:.2f} seconds")
    print(f"  GPU memory:        {result.memory_gb:.1f} GB")
else:
    print("No models available!")

### Multi-Model Benchmark Suite

In [ ]:
def run_benchmark_suite(models: List[str] = None) -> List[BenchmarkResult]:
    """
    Run benchmarks on multiple models.
    
    Args:
        models: List of model names (None = all available)
    
    Returns:
        List of BenchmarkResult objects
    """
    if models is None:
        models = available_models
    
    if not models:
        print("No models to benchmark!")
        return []
    
    results = []
    
    print("\n" + "=" * 70)
    print("                    DGX SPARK BENCHMARK SUITE")
    print("=" * 70)
    print(f"\nModels to test: {len(models)}")
    print(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    for i, model in enumerate(models, 1):
        print(f"\n[{i}/{len(models)}] Benchmarking: {model}")
        print("-" * 50)
        
        result = benchmark_model(model, num_runs=3)
        results.append(result)
        
        print(f"  ✅ {result}")
    
    return results

# Run suite on all available models
all_results = run_benchmark_suite()

---

## Part 5: Results Analysis

In [ ]:
def print_results_table(results: List[BenchmarkResult]):
    """Print results in a formatted table."""
    if not results:
        print("No results to display.")
        return
    
    print("\n" + "=" * 80)
    print("                         BENCHMARK RESULTS")
    print("=" * 80)
    print(f"{'Model':<30} {'Prefill (tok/s)':<18} {'Decode (tok/s)':<18} {'Memory (GB)':<12}")
    print("-" * 80)
    
    for r in results:
        print(f"{r.model:<30} {r.prefill_tps:<18.1f} {r.decode_tps:<18.1f} {r.memory_gb:<12.1f}")
    
    print("=" * 80)

print_results_table(all_results)

In [ ]:
# Expected results for comparison
EXPECTED_RESULTS = {
    "3b": {"prefill": 5000, "decode": 80, "memory": 3},
    "8b": {"prefill": 3000, "decode": 45, "memory": 6},
    "70b": {"prefill": 500, "decode": 15, "memory": 45},
}

def compare_with_expected(results: List[BenchmarkResult]):
    """Compare results with expected DGX Spark performance."""
    print("\n" + "=" * 80)
    print("                    COMPARISON WITH EXPECTED RESULTS")
    print("=" * 80)
    print("Note: Expected values are approximate and may vary with quantization.\n")
    
    for r in results:
        # Try to match model size
        model_lower = r.model.lower()
        expected = None
        
        for size_key, exp in EXPECTED_RESULTS.items():
            if size_key in model_lower:
                expected = exp
                break
        
        if expected:
            print(f"{r.model}:")
            
            # Prefill comparison
            prefill_pct = (r.prefill_tps / expected["prefill"]) * 100
            prefill_status = "✅" if prefill_pct >= 80 else "⚠️"
            print(f"  Prefill: {r.prefill_tps:.0f} tok/s (expected ~{expected['prefill']}) {prefill_status} {prefill_pct:.0f}%")
            
            # Decode comparison
            decode_pct = (r.decode_tps / expected["decode"]) * 100
            decode_status = "✅" if decode_pct >= 80 else "⚠️"
            print(f"  Decode:  {r.decode_tps:.0f} tok/s (expected ~{expected['decode']}) {decode_status} {decode_pct:.0f}%")
            
            # Memory comparison
            print(f"  Memory:  {r.memory_gb:.1f} GB (expected ~{expected['memory']} GB)")
            print()
        else:
            print(f"{r.model}: No expected results for comparison")
            print()

compare_with_expected(all_results)

---

## Part 6: Save Benchmark Results

In [ ]:
import json
from datetime import datetime

def save_results(results: List[BenchmarkResult], filename: str = None):
    """Save benchmark results to JSON file."""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"benchmark_results_{timestamp}.json"
    
    data = {
        "timestamp": datetime.now().isoformat(),
        "platform": "NVIDIA DGX Spark",
        "results": [
            {
                "model": r.model,
                "prompt_tokens": r.prompt_tokens,
                "generated_tokens": r.generated_tokens,
                "prefill_tokens_per_sec": r.prefill_tps,
                "decode_tokens_per_sec": r.decode_tps,
                "total_time_seconds": r.total_time_s,
                "memory_gb": r.memory_gb
            }
            for r in results
        ]
    }
    
    with open(filename, "w") as f:
        json.dump(data, f, indent=2)
    
    print(f"✅ Results saved to: {filename}")
    return filename

# Save results
if all_results:
    save_results(all_results)

In [ ]:
# Generate markdown report
def generate_markdown_report(results: List[BenchmarkResult]) -> str:
    """Generate a markdown report of benchmark results."""
    lines = [
        "# DGX Spark Ollama Benchmark Results\n",
        f"**Date:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n",
        f"**Platform:** NVIDIA DGX Spark (128GB Unified Memory)\n",
        "",
        "## Results\n",
        "| Model | Prefill (tok/s) | Decode (tok/s) | Memory (GB) |",
        "|-------|-----------------|----------------|-------------|"
    ]
    
    for r in results:
        lines.append(
            f"| {r.model} | {r.prefill_tps:.1f} | {r.decode_tps:.1f} | {r.memory_gb:.1f} |"
        )
    
    lines.extend([
        "",
        "## Notes\n",
        "- Prefill: Prompt processing speed (higher is better)",
        "- Decode: Token generation speed (higher is better)",
        "- Memory: GPU memory used by the model",
        "",
        "## Test Configuration\n",
        "- Prompt: 'Explain the concept of machine learning in simple terms.'",
        "- Max tokens: 100",
        "- Runs per model: 3 (averaged)"
    ])
    
    return "\n".join(lines)

# Generate and save report
if all_results:
    report = generate_markdown_report(all_results)
    
    with open("benchmark_report.md", "w") as f:
        f.write(report)
    
    print("✅ Report saved to: benchmark_report.md")
    print("\nPreview:")
    print(report[:500])

---

## Part 7: Different Prompt Lengths

In [ ]:
# Test with different prompt lengths
PROMPTS = {
    "short": "Hello!",
    "medium": "Explain the concept of machine learning in simple terms. What are the main types?",
    "long": """You are an expert AI researcher. Provide a comprehensive analysis of:
    1. The evolution of transformer architectures from 2017 to present
    2. Key innovations enabling billion-parameter models
    3. The role of attention in capturing dependencies
    4. Trade-offs between model size and efficiency
    5. Future directions for more efficient architectures
    Structure your response with clear sections and examples."""
}

def benchmark_prompt_lengths(model: str):
    """Benchmark a model with different prompt lengths."""
    print(f"\n{'='*60}")
    print(f"Prompt Length Analysis: {model}")
    print(f"{'='*60}")
    
    results = {}
    
    for name, prompt in PROMPTS.items():
        print(f"\nTesting '{name}' prompt ({len(prompt)} chars)...")
        result = benchmark_model(model, prompt=prompt, num_runs=2, warmup=False)
        results[name] = result
        print(f"  Prefill: {result.prefill_tps:.1f} tok/s, Decode: {result.decode_tps:.1f} tok/s")
    
    return results

# Run prompt length analysis on first available model
if available_models:
    prompt_results = benchmark_prompt_lengths(available_models[0])

---

## ⚠️ Common Mistakes

### Mistake 1: Measuring via web UI or CLI timing

```bash
# ❌ Wrong - Includes rendering overhead
time ollama run llama3.1:8b "Hello"

# ✅ Right - Use API for accurate timing
curl http://localhost:11434/api/generate -d '{
  "model": "llama3.1:8b",
  "prompt": "Hello",
  "stream": false
}'
```

### Mistake 2: Not doing warmup runs

```python
# ❌ Wrong - First run may be slow (model loading)
result = single_inference(model, prompt)

# ✅ Right - Warmup first
single_inference(model, "Hello", max_tokens=5)  # Warmup
result = single_inference(model, prompt)  # Real benchmark
```

### Mistake 3: Single run measurements

```python
# ❌ Wrong - High variance
result = benchmark(model)  # Single run

# ✅ Right - Average multiple runs
results = [benchmark(model) for _ in range(3)]
avg_speed = statistics.mean(r.decode_tps for r in results)
```

### Mistake 4: Not clearing buffer cache for large models

```bash
# ❌ Wrong - May OOM on 70B
ollama run llama3.1:70b

# ✅ Right - Clear cache first
sudo sh -c 'sync; echo 3 > /proc/sys/vm/drop_caches'
ollama run llama3.1:70b
```

---

## 🎉 Checkpoint

You've learned:
- ✅ The difference between prefill and decode metrics
- ✅ How to use Ollama's API for accurate benchmarking
- ✅ How to run multi-model benchmark suites
- ✅ How to compare results with expected performance
- ✅ How to save and report benchmark results

---

## 🚀 Challenge (Optional)

Create a benchmark that tests:
1. Different quantization levels (Q4, Q5, Q8, FP16)
2. Different context lengths (512, 2048, 8192 tokens)
3. Batch processing (multiple prompts at once)

<details>
<summary>💡 Hint</summary>

For quantization comparison:
```bash
ollama pull llama3.1:8b-q4_0
ollama pull llama3.1:8b-q8_0
```

For context length, use the `num_ctx` option:
```python
response = requests.post(url, json={
    "model": model,
    "prompt": long_prompt,
    "options": {"num_ctx": 8192}
})
```
</details>

In [ ]:
# YOUR CHALLENGE CODE HERE


---

## 📖 Further Reading

- [Ollama API Documentation](https://github.com/ollama/ollama/blob/main/docs/api.md)
- [LLM Inference Optimization](https://huggingface.co/docs/transformers/llm_optims)
- [NVIDIA DGX Spark Playbooks](https://build.nvidia.com/spark)

---

## 🧹 Cleanup

In [ ]:
print("Files created in this notebook:")
print("-" * 40)
print("  - benchmark_results_[timestamp].json")
print("  - benchmark_report.md")
print("\nThese are your benchmark records - keep them!")

print("\n" + "=" * 60)
print("🎉 Congratulations! You've completed Module 1!")
print("=" * 60)
print("\nYou now have a fully configured DGX Spark ready for AI development!")
print("\nNext: Module 1.2 - Python for AI/ML")
print("You'll learn Python best practices for high-performance AI code.")